In [ ]:
# ---
# title: 12. Advanced Portfolio Optimization Comparison
# tags: [Optimization, Benchmarking, Comparison, Finance]
# difficulty: Intermediate
# ---

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Import our optimizers
import sys
sys.path.append('..')
from src.analytics.advanced_optimizer import AdvancedOptimizer
from src.analytics.portfolio_optimizer import PortfolioOptimizer
from src.analytics.benchmark import BenchmarkEngine

sns.set_style('darkgrid')
plt.rcParams['figure.figsize'] = (14, 7)

# Advanced Portfolio Optimization: Comprehensive Comparison

## Introduction

This notebook provides a **side-by-side comparison** of all portfolio optimization techniques implemented in Market-Mind:

1. **Equal Weight** - Naive baseline (1/N allocation)
2. **Mean-Variance Optimization (MVO)** - Markowitz's Modern Portfolio Theory
3. **Hierarchical Risk Parity (HRP)** - Machine learning-based clustering approach
4. **Black-Litterman** - Bayesian framework combining market equilibrium with views
5. **CVaR Optimization** - Tail risk minimization

We'll evaluate each strategy across multiple dimensions:
- **Risk-Adjusted Returns**: Sharpe, Sortino, Calmar ratios
- **Tail Risk**: VaR, CVaR, Maximum Drawdown
- **Diversification**: Concentration metrics, effective number of assets
- **Stability**: Weight turnover, sensitivity to inputs

## Objective

By the end of this notebook, you'll understand:
- **When to use each optimization technique**
- **Trade-offs between different approaches**
- **How to select the right strategy for your investment goals**

## Step 1: Load Market Data

In [ ]:
# Load returns from silver layer
silver_path = Path("../data/silver")
files = list(silver_path.glob("market_returns_*.parquet"))

if not files:
    raise FileNotFoundError("No returns data found. Run data ingestion first.")

latest = max(files, key=lambda f: f.stat().st_mtime)
returns = pd.read_parquet(latest)

print(f"Loaded returns for {len(returns.columns)} tickers")
print(f"Date range: {returns.index[0]} to {returns.index[-1]}")
print(f"Total observations: {len(returns)}")

# Use a reasonable subset for demonstration
returns_subset = returns.iloc[:, :20]  # First 20 tickers
print(f"\nUsing {len(returns_subset.columns)} tickers for comparison")
print(f"Tickers: {', '.join(returns_subset.columns[:10].tolist())}...")

## Step 2: Compute Optimal Weights for All Strategies

In [ ]:
# Initialize optimizers
advanced_opt = AdvancedOptimizer()
mvo_opt = PortfolioOptimizer()

print("Computing optimal weights for all strategies...\n")

# 1. Equal Weight (Baseline)
equal_weights = pd.Series(1/len(returns_subset.columns), index=returns_subset.columns)
print("✅ Equal Weight")

# 2. Mean-Variance Optimization (Max Sharpe)
mvo_weights = mvo_opt.get_max_sharpe_weights(returns_subset)
print("✅ Mean-Variance Optimization (Max Sharpe)")

# 3. Hierarchical Risk Parity
hrp_weights = advanced_opt.get_hrp_weights(returns_subset)
print("✅ Hierarchical Risk Parity")

# 4. Black-Litterman (with example views)
# Create mock market caps
np.random.seed(42)
market_caps = pd.Series(np.random.dirichlet(np.ones(len(returns_subset.columns))), 
                        index=returns_subset.columns)

# Example views: bullish on first ticker, bearish on third
tickers = returns_subset.columns.tolist()
views = {tickers[0]: 0.03, tickers[2]: -0.02}
confidences = [0.7, 0.6]

bl_weights = advanced_opt.get_black_litterman_weights(
    returns_subset, market_caps=market_caps, 
    view_dict=views, confidences=confidences
)
print("✅ Black-Litterman")

# 5. CVaR Optimization
cvar_weights = advanced_opt.get_cvar_weights(returns_subset, alpha=0.95)
print("✅ CVaR Optimization")

# Combine all weights
all_weights = pd.DataFrame({
    'Equal Weight': equal_weights,
    'MVO (Max Sharpe)': mvo_weights,
    'HRP': hrp_weights,
    'Black-Litterman': bl_weights,
    'CVaR-Optimal': cvar_weights
}).fillna(0)

print("\n=== Weight Summary ===")
print(all_weights.describe())

## Step 3: Visualize Weight Allocations

In [ ]:
# Plot weight comparison
fig, axes = plt.subplots(2, 3, figsize=(18, 10))
axes = axes.flatten()

colors = ['gray', 'steelblue', 'coral', 'green', 'darkred']

for i, (col, color) in enumerate(zip(all_weights.columns, colors)):
    all_weights[col].sort_values(ascending=True).plot(
        kind='barh', ax=axes[i], color=color, alpha=0.8
    )
    axes[i].set_title(col, fontsize=12, fontweight='bold')
    axes[i].set_xlabel('Weight', fontsize=10)
    axes[i].grid(axis='x', alpha=0.3)

# Remove extra subplot
fig.delaxes(axes[5])

plt.tight_layout()
plt.show()

# Calculate concentration metrics
def herfindahl_index(weights):
    """Herfindahl-Hirschman Index (concentration)"""
    return (weights ** 2).sum()

def effective_n(weights):
    """Effective number of assets"""
    return 1 / herfindahl_index(weights)

concentration = pd.DataFrame({
    'Herfindahl Index': all_weights.apply(herfindahl_index),
    'Effective N': all_weights.apply(effective_n),
    'Max Weight': all_weights.max(),
    'Non-Zero Positions': (all_weights > 0.001).sum()
})

print("\n=== Portfolio Concentration Metrics ===")
print(concentration.to_string())
print("\nInterpretation:")
print("- Lower Herfindahl Index = More diversified")
print("- Higher Effective N = More evenly distributed weights")

## Step 4: Calculate Portfolio Returns

In [ ]:
# Calculate daily portfolio returns for each strategy
portfolio_returns = {}

for strategy in all_weights.columns:
    portfolio_returns[strategy] = (returns_subset * all_weights[strategy]).sum(axis=1)

# Convert to DataFrame
returns_df = pd.DataFrame(portfolio_returns)

# Calculate cumulative returns
cumulative_returns = (1 + returns_df).cumprod()

# Plot cumulative returns
plt.figure(figsize=(14, 7))
for col in cumulative_returns.columns:
    plt.plot(cumulative_returns.index, cumulative_returns[col], 
             label=col, linewidth=2, alpha=0.8)

plt.title('Cumulative Returns Comparison', fontsize=16, fontweight='bold')
plt.xlabel('Date', fontsize=12)
plt.ylabel('Cumulative Return', fontsize=12)
plt.legend(loc='best', fontsize=10)
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

print("\n=== Final Cumulative Returns ===")
final_returns = cumulative_returns.iloc[-1] - 1
print(final_returns.sort_values(ascending=False).to_string())

## Step 5: Comprehensive Performance Metrics

Using our `BenchmarkEngine` to calculate standardized metrics.

In [ ]:
# Initialize benchmark engine
benchmark = BenchmarkEngine(risk_free_rate=0.04)

# Calculate metrics for all strategies
all_metrics = []
for strategy, rets in portfolio_returns.items():
    metrics = benchmark.calculate_metrics(rets, name=strategy)
    all_metrics.append(metrics)

metrics_df = pd.DataFrame(all_metrics).set_index('Strategy')

print("\n=== Comprehensive Performance Metrics ===")
print(metrics_df.to_string())

# Format for better readability
print("\n=== Formatted Metrics ===")
formatted = metrics_df.copy()
for col in ['Total Return', 'Annualized Return', 'Volatility', 'Max Drawdown', 'Win Rate']:
    formatted[col] = formatted[col].apply(lambda x: f"{x:.2%}")
for col in ['Sharpe Ratio', 'Sortino Ratio', 'Calmar Ratio']:
    formatted[col] = formatted[col].apply(lambda x: f"{x:.2f}")
print(formatted.to_string())

## Step 6: Risk-Return Visualization

In [ ]:
# Create risk-return scatter plot
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Plot 1: Return vs Volatility (colored by Sharpe)
scatter = axes[0].scatter(
    metrics_df['Volatility'], 
    metrics_df['Annualized Return'],
    s=300, 
    c=metrics_df['Sharpe Ratio'],
    cmap='RdYlGn',
    alpha=0.7,
    edgecolors='black',
    linewidths=2
)

for idx, row in metrics_df.iterrows():
    axes[0].annotate(
        idx, 
        (row['Volatility'], row['Annualized Return']),
        fontsize=9, 
        ha='center',
        va='bottom'
    )

axes[0].set_xlabel('Volatility (Risk)', fontsize=12, fontweight='bold')
axes[0].set_ylabel('Annualized Return', fontsize=12, fontweight='bold')
axes[0].set_title('Risk-Return Trade-off', fontsize=14, fontweight='bold')
axes[0].grid(alpha=0.3)
plt.colorbar(scatter, ax=axes[0], label='Sharpe Ratio')

# Plot 2: Sharpe vs Sortino
axes[1].scatter(
    metrics_df['Sharpe Ratio'],
    metrics_df['Sortino Ratio'],
    s=300,
    alpha=0.7,
    c=range(len(metrics_df)),
    cmap='viridis',
    edgecolors='black',
    linewidths=2
)

for idx, row in metrics_df.iterrows():
    axes[1].annotate(
        idx,
        (row['Sharpe Ratio'], row['Sortino Ratio']),
        fontsize=9,
        ha='center',
        va='bottom'
    )

axes[1].set_xlabel('Sharpe Ratio', fontsize=12, fontweight='bold')
axes[1].set_ylabel('Sortino Ratio', fontsize=12, fontweight='bold')
axes[1].set_title('Risk-Adjusted Performance', fontsize=14, fontweight='bold')
axes[1].grid(alpha=0.3)

# Add diagonal reference line (Sharpe = Sortino)
lims = [0, max(metrics_df['Sortino Ratio'].max(), metrics_df['Sharpe Ratio'].max()) * 1.1]
axes[1].plot(lims, lims, 'k--', alpha=0.3, linewidth=1)

plt.tight_layout()
plt.show()

## Step 7: Drawdown Analysis

In [ ]:
# Calculate drawdowns for all strategies
def calculate_drawdown(returns_series):
    """Calculate drawdown series"""
    cum_returns = (1 + returns_series).cumprod()
    running_max = cum_returns.cummax()
    drawdown = (cum_returns - running_max) / running_max
    return drawdown

# Plot drawdowns
fig, axes = plt.subplots(2, 1, figsize=(14, 10))

# Subplot 1: All drawdowns
for strategy, rets in portfolio_returns.items():
    dd = calculate_drawdown(rets)
    axes[0].plot(dd.index, dd, label=strategy, linewidth=1.5, alpha=0.8)

axes[0].fill_between(dd.index, 0, dd, alpha=0.1)
axes[0].set_title('Drawdown Comparison', fontsize=14, fontweight='bold')
axes[0].set_ylabel('Drawdown', fontsize=12)
axes[0].legend(loc='lower left', fontsize=9)
axes[0].grid(alpha=0.3)
axes[0].axhline(0, color='black', linewidth=0.8)

# Subplot 2: Max drawdown comparison (bar chart)
metrics_df['Max Drawdown'].plot(kind='bar', ax=axes[1], color='darkred', alpha=0.7)
axes[1].set_title('Maximum Drawdown by Strategy', fontsize=14, fontweight='bold')
axes[1].set_ylabel('Max Drawdown', fontsize=12)
axes[1].set_xlabel('Strategy', fontsize=12)
axes[1].axhline(0, color='black', linewidth=0.8)
axes[1].grid(axis='y', alpha=0.3)
axes[1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

print("\n📊 Drawdown Insights:")
print(f"Best (smallest drawdown): {metrics_df['Max Drawdown'].idxmax()}")
print(f"Worst (largest drawdown): {metrics_df['Max Drawdown'].idxmin()}")

## Step 8: Strategy Rankings

In [ ]:
# Rank strategies by different metrics
ranking_metrics = [
    'Annualized Return', 'Sharpe Ratio', 'Sortino Ratio', 
    'Calmar Ratio', 'Max Drawdown', 'Win Rate'
]

rankings = pd.DataFrame()

for metric in ranking_metrics:
    # For Max Drawdown, higher (less negative) is better
    ascending = True if metric == 'Max Drawdown' else False
    rankings[metric] = metrics_df[metric].rank(ascending=ascending)

# Calculate average rank
rankings['Average Rank'] = rankings.mean(axis=1)
rankings = rankings.sort_values('Average Rank')

print("\n=== Strategy Rankings (1 = Best) ===")
print(rankings.to_string())

# Visualize rankings
fig, ax = plt.subplots(figsize=(12, 6))

rankings[ranking_metrics].plot(kind='bar', ax=ax, alpha=0.8)
ax.set_title('Strategy Rankings Across Metrics (Lower = Better)', 
             fontsize=14, fontweight='bold')
ax.set_ylabel('Rank', fontsize=12)
ax.set_xlabel('Strategy', fontsize=12)
ax.legend(title='Metric', bbox_to_anchor=(1.05, 1), loc='upper left')
ax.grid(axis='y', alpha=0.3)
ax.tick_params(axis='x', rotation=45)
plt.tight_layout()
plt.show()

print("\n🏆 Overall Winner (by average rank):")
print(f"   {rankings['Average Rank'].idxmin()} (Avg Rank: {rankings['Average Rank'].min():.2f})")

## Step 9: Correlation of Strategy Returns

In [ ]:
# Calculate correlation between strategy returns
strategy_corr = returns_df.corr()

# Plot correlation heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(strategy_corr, annot=True, fmt='.3f', cmap='coolwarm', 
            center=0, square=True, linewidths=1, cbar_kws={"shrink": 0.8})
plt.title('Correlation of Strategy Returns', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

print("\n📊 Correlation Insights:")
print("- High correlation (>0.9): Strategies behave similarly")
print("- Low correlation (<0.7): Potential diversification benefits")
print("\nLowest correlation pairs:")

# Find lowest correlation pairs
corr_pairs = []
for i in range(len(strategy_corr.columns)):
    for j in range(i+1, len(strategy_corr.columns)):
        corr_pairs.append((
            strategy_corr.columns[i],
            strategy_corr.columns[j],
            strategy_corr.iloc[i, j]
        ))

corr_pairs_df = pd.DataFrame(corr_pairs, columns=['Strategy 1', 'Strategy 2', 'Correlation'])
print(corr_pairs_df.sort_values('Correlation').head(3).to_string(index=False))

## Key Takeaways

### 📊 Strategy Comparison Summary

| Strategy | Best For | Strengths | Weaknesses |
|----------|----------|-----------|------------|
| **Equal Weight** | Simplicity, benchmarking | No estimation error, easy to implement | Ignores risk/return differences |
| **MVO (Max Sharpe)** | Maximizing risk-adjusted returns | Theoretically optimal, analytical solution | Unstable, concentrated, sensitive to inputs |
| **HRP** | Diversification, stability | Stable weights, no return estimates needed | Doesn't optimize returns explicitly |
| **Black-Litterman** | Incorporating views, institutional use | Flexible, stable, intuitive | Complex, requires market cap data |
| **CVaR-Optimal** | Tail risk management, downside protection | Focuses on worst-case losses | May sacrifice upside, computationally intensive |

### 🎯 Decision Framework

**Choose Equal Weight if:**
- You want a simple, transparent baseline
- Transaction costs are high
- You distrust optimization models

**Choose MVO if:**
- You have high-quality return forecasts
- You're comfortable with concentration
- You can rebalance frequently

**Choose HRP if:**
- Diversification is your primary goal
- Return estimates are unreliable
- You want stable, interpretable weights

**Choose Black-Litterman if:**
- You have subjective market views
- You want to respect market equilibrium
- You need explainable, stable portfolios

**Choose CVaR if:**
- Tail risk is your primary concern
- You're managing institutional capital with strict risk limits
- Downside protection matters more than upside capture

### 💡 Practical Recommendations

1. **Combine Strategies**: Consider ensemble approaches (e.g., average HRP and Black-Litterman weights)
2. **Regime-Dependent**: Switch strategies based on market conditions (HRP in high volatility, MVO in stable markets)
3. **Backtesting**: Always backtest on out-of-sample data before deploying
4. **Rebalancing**: Establish clear rebalancing rules (calendar-based or threshold-based)
5. **Transaction Costs**: Account for costs when comparing strategies

### 📚 Further Exploration

- **Risk Parity**: Allocate based on risk contribution (extension of HRP)
- **Robust Optimization**: Account for parameter uncertainty explicitly
- **Machine Learning**: Use ML for return prediction, then optimize
- **Multi-Period Optimization**: Consider dynamic rebalancing strategies

### 🔬 Next Steps

1. Run this comparison on different time periods
2. Test with different asset universes (international, multi-asset)
3. Incorporate transaction costs and constraints
4. Implement regime-switching framework
5. Build a meta-strategy that selects the best approach dynamically